In [ ]:
# Task: To build and compare predictive models using data. 
# Define a machine learning problem, including the input and the output and what they are used for

In [ ]:
# Single output (age), many input (humours) --> can we predict a person's age from the combination of humour they tend to use?
# Questionnaire results --> Model --> Age
# Classification (when age is discrete - use quantile binning) and 2x regression (when age is continuous)

## Dataset Construction and Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import seaborn as sns
from sklearn import preprocessing, linear_model
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
from sklearn.pipeline import Pipeline, make_pipeline
import copy

# Loading a tabular dataset from csv files using pandas
data = pd.read_csv("HSQ/data.csv")
# We can efficiently analyse all the data we have been given
# Our questionnaire acts as a sample for the whole population, so assume that this is true

In [2]:
data.describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q30,Q31,Q32,affiliative,selfenhancing,agressive,selfdefeating,age,gender,accuracy
count,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,...,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000
mean,2.025210,3.342670,3.078431,2.833800,3.599440,4.152194,3.277311,2.535014,2.582633,2.869281,...,3.945845,2.767507,2.838469,4.010644,3.375537,2.956583,2.762745,70.966387,1.455649,87.542484
std,1.075782,1.112898,1.167877,1.160252,1.061281,0.979315,1.099974,1.231380,1.224530,1.205013,...,1.135189,1.309601,1.233889,0.708479,0.661533,0.410870,0.645982,1371.989249,0.522076,12.038483
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,1.300000,0.000000,0.000000,0.000000,14.000000,0.000000,2.000000
25%,1.000000,3.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,...,3.000000,2.000000,2.000000,3.600000,2.900000,2.800000,2.300000,18.500000,1.000000,80.000000
50%,2.000000,3.000000,3.000000,3.000000,4.000000,4.000000,3.000000,2.000000,2.000000,3.000000,...,4.000000,3.000000,3.000000,4.100000,3.400000,3.000000,2.800000,23.000000,1.000000,90.000000
75%,3.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,3.000000,3.000000,4.000000,...,5.000000,4.000000,4.000000,4.500000,3.800000,3.300000,3.100000,31.000000,2.000000,95.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.100000,5.000000,5.000000,5.000000,44849.000000,3.000000,100.000000


In [ ]:
# We can look at the mean of each column and we can see that our average age is very skewed to the upper age limit
# Ensure we have removed invalid rows of data, with incorrect/missing entries (e.g. ages above 123)
# Remove any ages above 123 (which is the oldest recorded age rounded up) and 13 (which is the youngest legal age whose data you can store)
indexNames = data[(data["age"] > 123) | (data["age"] < 13)].index
data = data.drop(indexNames)
# Remove any rows with an accuracy of 0 (as these people don't want to be included in research) - not necessary seeing the described data
indexNames = data[(data["accuracy"] == 0)].index
data = data.drop(indexNames)
# Remove any rows where the questions weren't answered with an integer between -1 and 5 (as this is then corrupt data) - not necessary seeing the described data
for i in range(1, 32):
    indexNames = data[(data["Q" + str(i)] > 5) | (data["Q" + str(i)] < -1)].index
    data = data.drop(indexNames)
# Remove any rows where gender isn't an integer between 1 and 3
indexNames = data[(data["gender"] > 3) | (data["gender"] < 1)].index
data = data.drop(indexNames)

In [ ]:
# "Accuracy" data spread
data["accuracy"].describe()

In [ ]:
# "Accuracy" data plot
fig, axes = plt.subplots(figsize = (4, 4))
data["accuracy"].plot(kind = "kde").set_title("Accuracy")

In [ ]:
# We want accurate data whilst still having enough
# We could add a weighting according to each percentage, but these answer are subjective to the person that gave it
# Percentage errors are to do with accuracy. Remove results with an accuracy below 80% which is a comprimise between having enough data and accurate data [find reference]
indexNames = data[(data["accuracy"] < 80)].index
data = data.drop(indexNames)
print(data.shape)

In [ ]:
# Remove the categories (columns) of data we now aren't interested in
# Remove all the answers to the questions
data = data.drop(data.loc[:, "Q1":"Q32"].columns, axis = 1)
# Remove the gender category as we aren't looking into this
data = data.drop(["gender"], axis = 1)
data = data.drop(["accuracy"], axis = 1)

# Correct the spelling error of "agressive"
data.rename(columns = {"agressive":"aggressive"}, inplace = True)

print(data.head(10))

In [ ]:
# We have defined the input as the questionnaire humour values, and the output as the person's age
# Split the data into features x and target y
x = data.iloc[:, :-1].values
y = data["age"].values

In [ ]:
# Shuffle the data, but use a constant random seed so that we can compare fairly when testing different models
indices = np.arange(data.shape[0])
random_gen = np.random.RandomState(4112000) #This won't change since we have set the seed to stay the same
permutated = random_gen.permutation(indices)

In [ ]:
# How our data table looks so far
data.describe()

## Data Transformation

In [ ]:
# Plot humour styles
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 4))
data["affiliative"].plot(kind = "kde", ax = axes[0])
data["selfenhancing"].plot(kind = "kde", ax = axes[0])
data["aggressive"].plot(kind = "kde", ax = axes[0])
data["selfdefeating"].plot(kind = "kde", ax = axes[0])
axes[0].set_title('Questionnaire Results')

# Scale the data to have a mean and standard deviation of zero through standardisation, as we see all our variables follow a bell-shaped curve
# We could ideally compare with normalisation and see which works best
scaler = preprocessing.StandardScaler().fit(x)
x_scaled = scaler.transform(x)
x = x_scaled

# We want to test on the training set, pick the model that then does best on the validation set, and then confirm that this hold true for the test set
# Plot age
data["age"].plot(kind = "kde", ax = axes[1])
axes[1].set_title("Age")
y_scaled = (y - y.mean()) / y.std()
y = y_scaled

# Split the data into the training and test sets
# Dedicate 70% of the data to training - we have decided on a larger training set
# We want to ensure that each set is representative of the whole population
# Use a validation set to combat the issue of overfitting
training_size = round(0.7 * data.shape[0])
validation_size = round((data.shape[0] - training_size)/2)
testing_size = data.shape[0] - (training_size + validation_size)

# Consisting of affiliative, self-enhancing, agressive and self-defeating scores
x_training = x[permutated[:training_size]]
x_validation = x[permutated[training_size:(validation_size + training_size)]]
x_validation_training = x[permutated[:(training_size + validation_size)]]
x_testing = x[permutated[(validation_size + training_size):(validation_size + testing_size + training_size)]]

# Consisting of age corresponding to the correct four scores above
y_training = y[permutated[:training_size]]
y_validation = y[permutated[training_size:(validation_size + training_size)]]
y_validation_training = y[permutated[:(training_size + validation_size)]]
y_testing = y[permutated[(validation_size + training_size):(validation_size + testing_size + training_size)]]

In [ ]:
# If we treat age as ordinal categorical data
# Quantile binning age (when used for classification), since age is positively skewed
# This will ensure the frequency will be the same in each bracket
data_categorical = copy.deepcopy(data)
data_categorical["age_binned"], age_bins = pd.qcut(data_categorical["age"], q = 7, retbins = True)
data_categorical = data_categorical.drop(["age"], axis = 1)
print(age_bins)

# Print our bins
print(data_categorical.head(10))

# Transform each of these labels to numbers
data_categorical["age_binned"] = LabelEncoder().fit_transform(data_categorical["age_binned"])

# This data doesn't follow a bell-shaped curve, so normalise the binned age (instead standardising) to shift the values to center around zero
#data_categorical["age_binned"].plot(kind="kde").set_title("Binned Age")
#scaler2 = (y_training_c - y_training_c.min()) / (y_training_c.max() - y_training_c.min())
#y_scaled_c = (scaler2 * 2) - 1
y_c = data_categorical["age_binned"].values
scaler2 = (y_c - 3)
y_c = scaler2

# Redefine the y target values of the training, validation and testing sets, when we are using categorical data
# All the corresponding x values will stay the same
y_training_c = y_c[permutated[:training_size]]
y_validation_c = y_c[permutated[training_size:(validation_size + training_size)]]
y_validation_training_c = y_c[permutated[:(training_size + validation_size)]]
y_testing_c = y_c[permutated[(validation_size + training_size):(validation_size + testing_size + training_size)]]

In [ ]:
# How our transformed and cleaned data looks so far
print(data_categorical.head(5))
print(data.head(5))
print(data.describe)

## Linear Regression

In [ ]:
# Create the inital linear regression model
lin_regression = linear_model.LinearRegression()
lin_regression.fit(x_training, y_training)
# print(lin_regression.coef_)

# age_prediction = lin_regression.predict([[3, 2, 1, 4], [-1, 2, 1, 0]])
# print(age_prediction)
# Predict the validation set
lin_regression_predict = lin_regression.predict(x_validation)

print("MAE:", mean_absolute_error(y_validation, lin_regression_predict))
print("MSE:", mean_squared_error(y_validation, lin_regression_predict))

In [ ]:
# No hyperparameter tuning necessary
print(lin_regression.get_params(deep=True).items())

In [ ]:
# Create this model, fitting the other 80% of the data
lin_regression2 = linear_model.LinearRegression()
lin_regression.fit(x_validation_training, y_validation_training)

# Apply the model to the testing data to predict
lin_regression_predict2 = lin_regression.predict(x_testing)

In [ ]:
#print(len(x_training), len(lin_regression_predict), x_training, lin_regression_predict)
# Visualisation for each feature
x_testing_col1 = [row[0] for row in x_testing]
x_testing_col2 = [row[1] for row in x_testing]
x_testing_col3 = [row[2] for row in x_testing]
x_testing_col4 = [row[3] for row in x_testing]

# We get graphs with a predicted plane, since we don't have 2D data, hence there isn't a single line of best fit
plt.suptitle("Transformed Age Preditions (Affiliative Humour)", fontsize = 14)
plt.xlabel("Affiliative Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_testing_col1, y_testing, color = "green")
plt.scatter(x_testing_col1, lin_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Self-Enhancing Humour)", fontsize = 14)
plt.xlabel("Self-Enhancing Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_testing_col2, y_testing, color = "green")
plt.scatter(x_testing_col2, lin_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Aggressive Humour)", fontsize = 14)
plt.xlabel("Aggressive Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_testing_col3, y_testing, color = "green")
plt.scatter(x_testing_col3, lin_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Self-Defeating Humour)", fontsize = 14)
plt.xlabel("Self-Defeating Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_testing_col4, y_testing, color = "green")
plt.scatter(x_testing_col4, lin_regression_predict2, color = "red")
plt.show()

In [ ]:
# Allows us to see how good our current linear model is
print("MAE:", mean_absolute_error(y_testing, lin_regression_predict2))
print("MSE:", mean_squared_error(y_testing, lin_regression_predict2))

## Polynomial Regression

In [ ]:
# Create the inital polynomial regression model
degree = PolynomialFeatures(degree = 6)
x_poly_training = degree.fit_transform(x_training)
poly_regression = linear_model.LinearRegression()
poly_regression.fit(x_poly_training, y_training)

# Predict the validation set
x_poly_validation = degree.fit_transform(x_validation)
poly_regression_predict = poly_regression.predict(x_poly_validation)

print("MAE:", mean_absolute_error(y_validation, poly_regression_predict))
print("MSE:", mean_squared_error(y_validation, poly_regression_predict))

In [ ]:
# Hyperparameter tuning with polynomial degree
degree.get_params(deep=True).items()

# The parameters we want to tune
degree2 = [{"poly__degree": list(range(2, 3))}]

# Create a new empty model and transform the data
pipeline = Pipeline(steps=[('poly', PolynomialFeatures()), ('model', linear_model.LinearRegression())])

# Use cross-validation (using the validation set separated 4 times (with the fifth section being the test data)) and Grid Search
#cv = RepeatedKFold(n_splits = 4, n_repeats = 4, random_state = 1)
search = GridSearchCV(pipeline, degree2, scoring = "neg_root_mean_squared_error")#, cv = cv)

# Fit the best values for the hyperparameters for the model. Use the training data
best_poly = search.fit(x_poly_training, y_training)

# Print the hyperparameters
print("Degree:", best_poly.best_estimator_.get_params()["poly__degree"])

In [ ]:
# Define the new best model
degree3 = PolynomialFeatures(degree = best_poly.best_estimator_.get_params()["poly__degree"])
x_poly_validation_training = degree3.fit_transform(x_validation_training)
poly_regression3 = linear_model.LinearRegression()
poly_regression3.fit(x_poly_validation_training, y_validation_training)

x_poly_testing = degree3.fit_transform(x_testing)
poly_regression_predict2 = poly_regression3.predict(x_poly_testing)

In [ ]:
# Print graph similarly to linear regression, to help us visualise our model
# Visualise for each feature
# We have to refer to different columns when using .fit_transform
x_poly_testing_col1 = [row[1] for row in x_poly_testing]
x_poly_testing_col2 = [row[2] for row in x_poly_testing]
x_poly_testing_col3 = [row[3] for row in x_poly_testing]
x_poly_testing_col4 = [row[4] for row in x_poly_testing]
#x_poly_testing_col15 = [row[14] for row in x_poly_testing]

# We get graphs with a predicted plane, since we don't have 2D data, hence there isn't a single line of best fit
plt.suptitle("Transformed Age Preditions (Affiliative Humour)", fontsize = 14)
plt.xlabel("Affiliative Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_poly_testing_col1, y_testing, color = "green")
plt.scatter(x_poly_testing_col1, poly_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Self-Enhancing Humour)", fontsize = 14)
plt.xlabel("Self-Enhancing Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_poly_testing_col2, y_testing, color = "green")
plt.scatter(x_poly_testing_col2, poly_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Aggressive Humour)", fontsize = 14)
plt.xlabel("Aggressive Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_poly_testing_col3, y_testing, color = "green")
plt.scatter(x_poly_testing_col3, poly_regression_predict2, color = "red")
plt.show()

plt.suptitle("Transformed Age Preditions (Self-Defeating Humour)", fontsize = 14)
plt.xlabel("Self-Defeating Humour", fontsize = 10)
plt.ylabel("Age", fontsize = 12)
plt.scatter(x_poly_testing_col4, y_testing, color = "green")
plt.scatter(x_poly_testing_col4, poly_regression_predict2, color = "red")
plt.show()

In [ ]:
# Allows us to see how good our current polynomial model is
print("MAE:", mean_absolute_error(y_testing, poly_regression_predict2))
print("MSE:", mean_squared_error(y_testing, poly_regression_predict2))

## K-Nearest Neighbours

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 6, leaf_size = 20, p = 1)
knn.fit(x_training, y_training_c)

# Predict validation set
knn_predict = knn.predict(x_validation)

print("Confusion matrix:")
print(confusion_matrix(y_validation_c, knn_predict))
print("Accuracy score:", accuracy_score(y_testing_c, knn_predict))
print("Classification report:", classification_report(y_validation_c, knn_predict))
# Each line of our classification report refers to each bin we previously defined

In [ ]:
# Hyperparameter tuning with leaf size (the maximum number of points belonging to each leaf node) and the number of neigbors (classifications)
print(knn.get_params(deep=True).items())

# Create a new empty model
knn2 = KNeighborsClassifier()

# The parameters we want to tune
leaf_size = list(range(1, 50))
neighbors = list(range(1, 30))
p = [1, 2]
metric = ["minkowski", "hamming", "manhattan", "euclidean"]

# Converting the parameters to a dictionary
hyperparameters = dict(leaf_size = leaf_size, n_neighbors = neighbors, p = p, metric = metric)

# Use cross-validation (using the validation set separated 4 times (with the fifth section being the test data)) and Grid Search
cv = RepeatedStratifiedKFold(n_splits = 4, n_repeats = 4, random_state = 1)
search = GridSearchCV(knn2, hyperparameters, scoring = "accuracy", cv = cv)

# Fit the best values for the hyperparameters for the model. Use the training data
best_knn = search.fit(x_training, y_training_c)

# Print the hyperparameters
print("Best leaf_size:", best_knn.best_estimator_.get_params()['leaf_size'])
print("Best p:", best_knn.best_estimator_.get_params()['p'])
print("Best n_neighbors:", best_knn.best_estimator_.get_params()['n_neighbors'])
print("Best metric:", best_knn.best_estimator_.get_params()["metric"])

In [ ]:
# Define the new best model
knn3 = KNeighborsClassifier(n_neighbors = best_knn.best_estimator_.get_params()["n_neighbors"], leaf_size = best_knn.best_estimator_.get_params()["leaf_size"], p = best_knn.best_estimator_.get_params()["p"], metric = best_knn.best_estimator_.get_params()["metric"])
knn3.fit(x_validation_training, y_validation_training_c)

knn_predict2 = knn3.predict(x_testing)

In [ ]:
# Allows us to see how good our current knn model is
print("Confusion matrix:")
print(confusion_matrix(y_testing_c, knn_predict2))
print("Accuracy score:", accuracy_score(y_testing_c, knn_predict2))
print("Classification report:")
print(classification_report(y_testing_c, knn_predict2))

In [ ]:
# NOTE: For polynomial regression, the cross-validation code is commented out, since it was taking long to run on my device
# I also changed the hyperparameter polynomial degree from range(2, 4) to range(2, 3) since it was overloading my device